# Bertopic modeling

In [1]:
# !pip install pyarrow==15.0.2 bertopic datasets accelerate bitsandbytes xformers adjustText
# required lib
from collections import Counter
import pandas as pd
import pickle
import re
import string
import tqdm

In [ ]:
# our file is in json format, you can replace with your own file location and format
df = pd.read_json('xxxx', compression='gzip', lines=True) # for privacy reason, I deleted the URL
df.head()

,author,created_utc,id,selftext,subreddit,title,date,crime,false_match
0,[deleted],1546314346,abemmi,[deleted],Atlanta,Gunshots?,2019-01-01 03:45:46,True,False
1,[deleted],1546314372,abemqc,[removed],Atlanta,Anybody hear gunshots?,2019-01-01 03:46:12,True,False
2,[deleted],1546380318,abmjmw,[deleted],Atlanta,"Shot taken on a hazy, in-town hike a couple da...",2019-01-01 22:05:18,True,False
3,WildVelociraptor,1546439160,abtu2p,,Atlanta,Gwinnett County teen commits suicide after acc...,2019-01-02 14:26:00,True,False
4,Downbad2018,1546482755,ac0vud,I'm in the process of leaving Charlotte and I ...,Atlanta,Material difference between Charlotte and Atlanta,2019-01-03 02:32:35,True,False


In [10]:
# Concatenate and clean text, with a colon after the title
def concatenate_and_clean(row):
    # Concatenate title and selftext with a colon after the title
    concatenated_text = f"{row['title']}: {row['selftext']}"
    # Remove unwanted phrases
    cleaned_text = concatenated_text.replace('[removed]', '').replace('[deleted]', '').strip()
    return cleaned_text

# Apply the function to the dataframe to create a new 'text' column
df['text'] = df.apply(concatenate_and_clean, axis=1)
df.head()

,author,created_utc,id,selftext,subreddit,title,date,crime,false_match,text
0,[deleted],1546314346,abemmi,[deleted],Atlanta,Gunshots?,2019-01-01 03:45:46,True,False,Gunshots?:
1,[deleted],1546314372,abemqc,[removed],Atlanta,Anybody hear gunshots?,2019-01-01 03:46:12,True,False,Anybody hear gunshots?:
2,[deleted],1546380318,abmjmw,[deleted],Atlanta,"Shot taken on a hazy, in-town hike a couple da...",2019-01-01 22:05:18,True,False,"Shot taken on a hazy, in-town hike a couple da..."
3,WildVelociraptor,1546439160,abtu2p,,Atlanta,Gwinnett County teen commits suicide after acc...,2019-01-02 14:26:00,True,False,Gwinnett County teen commits suicide after acc...
4,Downbad2018,1546482755,ac0vud,I'm in the process of leaving Charlotte and I ...,Atlanta,Material difference between Charlotte and Atlanta,2019-01-03 02:32:35,True,False,Material difference between Charlotte and Atla...


In [11]:
# Check if the text is less than 5 characters or contains only a URL
def is_short_or_url(text):
    # Check if the text has less than 5 characters
    if len(text) < 5:
        return True
    # Check if the text contains only a URL
    if re.match(r'^https?://\S+$', text):
        return True
    return False

# Apply the function to the 'text' column
df['short_or_url'] = df['text'].apply(is_short_or_url)


# Filter the rows where the condition is True
filtered_df = df[df['short_or_url']]
print('dropping %d rows that are < 5 chars or just a url' % len(filtered_df))
df = df[~df['short_or_url']].drop(columns=['short_or_url'])
df.shape

dropping 82 rows that are < 5 chars or just a url


(51429, 10)

In [12]:
# Skipping this step. -awc
# Remove duplicate values.
# df.drop_duplicates(subset='text', inplace=True)
# df.shape

In [13]:
# function taken and modified
# from https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76

def simple_text_clean(x):
    # first we lowercase everything
    #x = x.lower()

    # remove unicode characters
    #x = x.encode('ascii', 'ignore').decode() # not for finnish

    # remove @ symbols and hashtags
    #x = re.sub(r'@\S', '', x)
    #x = re.sub(r'#\S+', ' ', x)
    x = re.sub('([#])','',x )
    x = re.sub('([@])','',x )
    x = re.sub('^[RT]+','',x ) # delete RT

    # remove words less then 2 characters
    x = re.sub(r'\b\w{1,1}\b', '', x)

    # Remove URL
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'http*\S+', ' ', x)

    # Remove ticks and the next character
    x = re.sub("\'\w+", '', x)
    # Remove punctuations
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    # Remove numbers
    x = re.sub(r'\w*\d+\w*', '', x)
    # Replace the over spaces
    x = re.sub('\s{2,}', " ", x)
    #x = re.sub(r'\s[^\w\s]\s', '', x)

    return x

In [14]:
df['cleaned_text'] = df.text.apply(simple_text_clean)

# remove not only empty strings, but also strings only containing whitespace
# https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
df = df[df['cleaned_text'].str.strip().astype(bool)]
df.shape

(51429, 11)

In [15]:
# Skipping this step. -awc
# # Remove duplicate values
# df.drop_duplicates(subset='cleaned_text', inplace=True)
# df.shape

In [16]:
pd.set_option('display.max_colwidth', None)
df.cleaned_text.head(5)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Sentence Embedded

In [ ]:
# Bert needs text in list
tweet_list = df.cleaned_text.to_list()
timestamp = df.date.to_list() # if you have a date stamp, use it here for timeline analysis

In [ ]:
# The embedding model we will use
from sentence_transformers import SentenceTransformer

# Prepare embeddings
# models could vary based on data sets, check out huggingface
# since we analyze short text, we used this model but even this is now outdated

sentence_model = SentenceTransformer("BAAI/bge-small-en")

# you can also use normal embeddings, as well as BERT type transformers
# check BERTopic github page for details

/Users/qge5yv/miniforge3/envs/nlp/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Since we will use embedding models several times until we find a satisfactory results, it is wise to create embeddings in advance
# Else bertopic will do the same process repeatedly
# this saves time

embeddings = sentence_model.encode(tweet_list, show_progress_bar=True)

Batches:   0%|          | 0/1608 [00:00<?, ?it/s]

In [18]:


# #Store sentences & embeddings on disc
with open('topic/embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': tweet_list,'timestamp':timestamp, 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# if you want to continue with embeddings on the memory, use these
stored_sentences = tweet_list
stored_timestamp = timestamp
stored_embeddings = embeddings

# Model

## Embedded data

In [ ]:
# if you turn back to BERTopic after above stage, you can start from here

import pickle

#Load sentences & embeddings from disc
with open('topic/embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_timestamp = stored_data['timestamp']
    stored_embeddings = stored_data['embeddings']

In [ ]:
# make sure about data size
print('embedding size:',len(stored_embeddings))
print(f'Timestamp Size: {len(stored_timestamp)}')
print('tweet size:',len(stored_sentences))

embedding size: 51429
Timestamp Size: 51429
tweet size: 51429


In [4]:
# The embedding model we will use
from sentence_transformers import SentenceTransformer

# Prepare embeddings
sentence_model = SentenceTransformer("BAAI/bge-small-en")


/Users/qge5yv/miniforge3/envs/nlp/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Bertopic

In [ ]:
import numpy as np
import pandas as pd

from bertopic import BERTopic


from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from sklearn.decomposition import PCA

from hdbscan import HDBSCAN

from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
# for the local computer warning
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # normally it is false
#print(stopwords.words('english'))

# Stopwords
stoplist = list(stopwords.words('english')) + ['http', 'https', 'amp', 'com', 'co'] # you can include as much as you want
# I suggest to add words and terminologies if you are working on specific topic
# for example, if you analyze how LLM is used in education, better to delete LLM, ai, chatgpt, chatbox type words because they more or less appear in all texts

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qge5yv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration


# these models are useful to extract different representative words from topics
# Keybert and MMR are more useful compared to default model results
# Instead of words, LLMs provide description about your topics

# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# if you are using cloud server, you can include LLM models
# check details in BERTopic github page

# Text generation with Llama 3
#llama3 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    #"Llama2": llama3,
    "MMR": mmr,
}

In [7]:
import random

# Set random seed
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)

# https://towardsdatascience.com/basics-of-countvectorizer-e26677900f9c
# max_df is used for removing terms that appear too frequently, also known as "corpus-specific stop words".
# max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
# max_df = 25 means "ignore terms that appear in more than 25 documents".

#min_df is used for removing terms that appear too infrequently.
#min_df = 0.01 means "ignore terms that appear in less than 1% of the documents".
#min_df = 5 means "ignore terms that appear in less than 5 documents".

#The CountVectorizer will select the words/features/terms which occur the most frequently.

vectorizer_model = CountVectorizer(ngram_range=(1, 3),
                                   stop_words=stoplist,
                                   max_features=10000,
                                   min_df= 0.01,
                                   max_df= 0.90
                                   )

#umap models
# lower values of n_neighbors will focus more on the very local structure
# Increasing this value often results in larger clusters being created.

# n_components parameter controls the dimensionality of the final embedded data
# Increasing this value too much and HDBSCAN will have a hard time clustering the high-dimensional embeddings.
# min_dist= We will look at a range of values from 0.0 through to 0.99.

umap_model = UMAP(n_neighbors=40,
                  n_components=5, # default 5
                  min_dist=0.1,    # default 0.1
                  metric='cosine',
                 random_state= 42)
#dim_model = PCA(n_components=5)


In [17]:
# Skip dimensionality reduction
from bertopic import BERTopic
from bertopic.dimensionality import BaseDimensionalityReduction



# HDBSCAN model
# min_cluster_size is arguably the most important parameter in HDBSCAN
# it controls the smallest grouping you want to consider as a cluster
# Increasing this value results in less clusters but of larger size
# whereas decreasing this value results in more micro clusters being generated.
# Typically, I would advise on increasing this value rather than decreasing it.

# min_samples is automatically set to min_cluster_size and controls the amount of outliers are generated.
# defaults to being equal to min_cluster_size if unspecified,
# Setting this value significantly lower than min_cluster_size
# might help you reduce the amount of noise you will get..

hdbscan_model = HDBSCAN(min_cluster_size=80, # default 10
                        min_samples=4, #The larger it is, the more points are discarded as noise/outliers.
                        #cluster_selection_epsilon= 0.5,# separate clusters that are less than 0.5 units apart
                        metric='euclidean',
                        cluster_selection_method='eom',
                        prediction_data=True)

random.seed(seed_value)
# Fit BERTopic without actually performing any dimensionality reduction
topic_model = BERTopic(
                       embedding_model=sentence_model, # if not using embedding model
                       vectorizer_model=vectorizer_model,
                       hdbscan_model=hdbscan_model,
                       top_n_words=10,
                       min_topic_size=15, # larger value, broader document coverage
                       #nr_topics="auto",
                       calculate_probabilities= True,
                       verbose=True,
                       # need this to skip dimensionality reduction; otherwise reduces to 5 components using umap by default.
                       #  umap_model=BaseDimensionalityReduction(),
                       # but, these results are not very good (half in cluster 0)
                       representation_model=representation_model)


#topics, probabilities = topic_model.fit_transform(stored_sentences, stored_embeddings)
topics, probs = topic_model.fit_transform(stored_sentences, stored_embeddings)
df['topic'] = topics

2024-09-20 17:40:39,888 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-20 17:40:48,923 - BERTopic - Dimensionality - Completed ✓
2024-09-20 17:40:48,924 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-09-20 17:41:01,311 - BERTopic - Cluster - Completed ✓
2024-09-20 17:41:01,316 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-09-20 17:41:16,745 - BERTopic - Representation - Completed ✓


In [18]:
df.topic.value_counts(normalize=False)

topic
-1     20194
 0      3483
 1      1987
 2      1433
 3      1191
       ...  
 88       89
 89       87
 90       83
 91       83
 92       80
Name: count, Length: 94, dtype: int64

In [19]:
# lots of -1 docs; and they look pretty relevant
[i[:100] for i in df[df.topic==-1].sample(n=50).text.values]

["Pres Trump backs Int'l Soros child rape ring for $4billion bribe Pelosi rec'd $3b 'to ensure safe pa",
 'Sources: Man accused of shooting two St. Louis police officers has a lengthy criminal history:',
 'Neighbor calls cops on bipolar queens man for gun possession (?). Cops find no gun but taser man to ',
 'I need a new neighborhood. Priced out of mine.: I’ve lived in BV since I moved to mke in 2002 and I ',
 'Murder charges filed in Logan Square fatal shooting last month, police say:',
 'Cook County Jail inmate charged with murder in beating death of cellmate:',
 '31 year prison sentence for Lovelle Jordan. Initiated shootout with police outside 25th District sta',
 'Two Jewish Men shot in Pico-Robertson - Two Days in a row:',
 'Houstonians... please help me with this neighborhood...: I have found a job in Houston. For next 6 -',
 'Seven bodies, nothing stolen: Were killings at Riverside marijuana grow ‘a message’?:',
 'McKinley Park Neighborhood Crime Rises 34 Percent in 2022 Over 

In [ ]:
# we enforce to distribute texts across topics with defined theshold level
# it is not always necessary, but sometimes useful
# outlier reduction
new_topics = topic_model.reduce_outliers(stored_sentences, topics, strategy="c-tf-idf", embeddings=stored_embeddings, threshold=0.1)
print('%d outliers remain' % Counter(new_topics)[-1])

3927 outliers remain


In [ ]:
# we are comparing new distribution of topics
topic_diffs = pd.DataFrame([Counter(topics), Counter(new_topics)], index=['old', 'new']).T
topic_diffs['diff'] = topic_diffs.new-topic_diffs.old
topic_diffs.sort_values('diff', ascending=False).head(20)

,old,new,diff
2,1433,2105,672
11,613,1201,588
31,245,776,531
20,370,896,526
30,254,710,456
17,531,930,399
69,121,475,354
61,142,487,345
59,146,486,340
65,129,458,329


In [ ]:
# update topics
topic_model.update_topics(stored_sentences, topics=new_topics)
df['topic'] = new_topics

2024-09-20 17:43:46,557 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [23]:
# Check the type and length of topics
print("Type of topics:", type(topics))  # Should be a list or numpy array
print("Length of topics:", len(topics))  # Should match the length of stored_sentences


Type of topics: <class 'list'>
Length of topics: 51429


In [24]:
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq


Number of topics: 94


Topic  Count                                   Name  \
0      -1   3927                  -1_shooting_in_on_the   
1       0   3545                   0_shot_shots_film_of   
2       1   2184      1_gunshots_anyone_fireworks_heard   
3       2   2105              2_shooting_shot_dead_mass   
4       3   1509              3_assault_sexual_rape_sex   
..    ...    ...                                    ...   
89     88    118       88_scooter_moped_stolen_scooters   
90     89    140          89_orleans_new_quarter_french   
91     90    132  90_carjackings_chicago_carjacking_cwb   
92     91    247           91_violent_crime_mayor_total   
93     92    179                 92_orleans_new_you_the   

                                                                       Representation  \
0                               [shooting, in, on, the, shot, at, for, to, crime, of]   
1                           [shot, shots, film, of, this, from, drone, some, the, my]   
2          [gunshots, anyone, fireworks, heard, shooting, hear, was, on, what, shots]   
3               [shooting, shot, dead, mass, injured, park, man, killed, wounded, at]   
4         [assault, sexual, rape, sex, man, raped, woman, charged, accused, arrested]   
..                                                                                ...   
89            [scooter, moped, stolen, scooters, it, vespa, my, riding, electric, if]   
90     [orleans, new, quarter, french, bourbon, shot, nopd, police, shooting, street]   
91  [carjackings, chicago, carjacking, cwb, armed, police, spree, robberies, een, in]   
92        [violent, crime, mayor, total, detroit, fbi, rate, cities, city, homicides]   
93                                [orleans, new, you, the, to, that, it, and, we, of]   

                                                                                                                                                    KeyBERT  \
0                                                                     [community, chicago, neighborhood, home, nyc, public, back, houston, street, memphis]   
1                                               [shot film, film shot, another shot, video shot, shots, shot took, shot, favorite shots, shoot, drone shot]   
2                    [heard gunshots, hear gunshots, gunshots fireworks, gunshots, heard shots, shootings, gunshot, shots fired, anyone else hear, gunfire]   
3                    [shooting, shooting shooting, shooting near, shootings, another shooting, gunshots, shooting outside, people shot, shot near, shooter]   
4   [sexual assault, accused sexual assault, sexually assault, sex assault, assaulted, accused sexual, charged assault, assault charges, assaults, assault]   
..                                                                                                                                                      ...   
89                                           [scooter stolen, stolen, scooter, stolen today, got stolen, theft, scooters, get stolen, moped, police report]   
90         [new orleans police, shot killed, people shot, killed shooting, orleans police, fatally shot, new orleans, shot dead, woman shot, shooting shot]   
91              [carjackings, chicago crime, carjacking, armed carjacking, chicago police, chicago police say, carjack, chicago, across chicago, robberies]   
92                          [violent crime rate, detroit police, violent crime, city crime, crime rate, crime, crime spike, crime new, detroit, crime plan]   
93                                                [new orleans, katrina, nola, louisiana, orleans, politicians, hurricanes, hurricane, memphis, mardi gras]   

                                                                                                                                                      MMR  \
0                                                                              [crime, like, shooting, murder, chicago, stolen, school, nyc, years, home]   
1  

In [ ]:
# let's look at the extracted words in topic 1
topic_model.get_topic(1)

[('gunshots', np.float64(0.022669395235948128)),
 ('anyone', np.float64(0.01830098736062849)),
 ('fireworks', np.float64(0.017949794179444485)),
 ('heard', np.float64(0.017265067021071053)),
 ('shooting', np.float64(0.013858283398165389)),
 ('hear', np.float64(0.013678860469208489)),
 ('was', np.float64(0.012750080035884903)),
 ('on', np.float64(0.011429623339518625)),
 ('what', np.float64(0.011420309975278974)),
 ('shots', np.float64(0.011174566078856338))]

In [26]:
# file_path = "/content/gdrive/My Drive/Current Projects/Tulane/"
!mkdir -p ./figures
file_path="./"

# Generate the figure using BERTopic
bar_topics = topic_model.visualize_barchart(top_n_topics=90)
#bar_topics.write_html(file_path+"figures/bartopics.html")
# Save the file by concatenating the file path and the file name
bar_topics.write_html(file_path + "figures/bartopics.html")
bar_topics

In [27]:
# Generate the figure using BERTopic
topic_distance = topic_model.visualize_topics()
#topic_distance.write_html("figures/topicdistance.html")
topic_distance.write_html(file_path + "figures/topicdistance.html")
topic_distance

In [28]:

topic_heatmap = topic_model.visualize_heatmap()
topic_heatmap.write_html(file_path + "figures/topic_heatmap.html")
topic_heatmap

In [29]:

# Generate the figure using BERTopic
topic_hierarchy = topic_model.visualize_hierarchy()
topic_hierarchy.write_html(file_path + "figures/topic_hierarchy.html")
topic_hierarchy

## Representative Posts

In [ ]:
# # Store topics in a new variable to avoid overwriting
# topic_assignments = topics.copy()
# # Create the DataFrame
# posts = pd.DataFrame({'topic': topic_assignments, 'document': stored_sentences})


In [ ]:
# posts

In [ ]:
# topic_0 = posts[posts.topic == 0]
# topic_0

In [30]:
topic_labels = topic_model.generate_topic_labels(nr_words=3,
                                                 topic_prefix=False,
#                                                  word_length=10,
                                                 separator=", ")
topic_labels

['shooting, in, on',
 'shot, shots, film',
 'gunshots, anyone, fireworks',
 'shooting, shot, dead',
 'assault, sexual, rape',
 'stolen, thief, theft',
 'nypd, shot, brooklyn',
 'baltimore, police, in',
 'car, stolen, my',
 'dog, cat, dogs',
 'chicago, police, shot',
 'stabbed, death, man',
 'me, was, he',
 'hate, asian, crime',
 'vaccine, covid, governor',
 'old, year, shot',
 'we, you, and',
 'chicago, violence, crime',
 'crime, rate, is',
 'protest, protests, the',
 'york, nyc, new',
 'area, is, to',
 'houston, hpd, shot',
 'prosecutors, cwb, chicago',
 'carjacking, carjackings, attempted',
 'gov, vote, election',
 'gun, ranges, shoot',
 'bike, stolen, bikes',
 'stolen, car, vehicle',
 'bike, bikes, stolen',
 'gun, guns, store',
 'robbery, armed, gunpoint',
 'murder, suicide, murders',
 'subway, brooklyn, station',
 'armed, robberies, lincoln',
 'deputy, sheriff, deputies',
 'podcast, murder, true',
 'they, to, the',
 'catalytic, converter, theft',
 'louis, st, county',
 'officer, po

In [31]:
# Number of topics
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))

display(freq)
# Head
#freq[1:21]


Number of topics: 94


Topic  Count                                   Name  \
0      -1   3927                  -1_shooting_in_on_the   
1       0   3545                   0_shot_shots_film_of   
2       1   2184      1_gunshots_anyone_fireworks_heard   
3       2   2105              2_shooting_shot_dead_mass   
4       3   1509              3_assault_sexual_rape_sex   
..    ...    ...                                    ...   
89     88    118       88_scooter_moped_stolen_scooters   
90     89    140          89_orleans_new_quarter_french   
91     90    132  90_carjackings_chicago_carjacking_cwb   
92     91    247           91_violent_crime_mayor_total   
93     92    179                 92_orleans_new_you_the   

                                                                       Representation  \
0                               [shooting, in, on, the, shot, at, for, to, crime, of]   
1                           [shot, shots, film, of, this, from, drone, some, the, my]   
2          [gunshots, anyone, fireworks, heard, shooting, hear, was, on, what, shots]   
3               [shooting, shot, dead, mass, injured, park, man, killed, wounded, at]   
4         [assault, sexual, rape, sex, man, raped, woman, charged, accused, arrested]   
..                                                                                ...   
89            [scooter, moped, stolen, scooters, it, vespa, my, riding, electric, if]   
90     [orleans, new, quarter, french, bourbon, shot, nopd, police, shooting, street]   
91  [carjackings, chicago, carjacking, cwb, armed, police, spree, robberies, een, in]   
92        [violent, crime, mayor, total, detroit, fbi, rate, cities, city, homicides]   
93                                [orleans, new, you, the, to, that, it, and, we, of]   

                                                                                                                                                    KeyBERT  \
0                                                                     [community, chicago, neighborhood, home, nyc, public, back, houston, street, memphis]   
1                                               [shot film, film shot, another shot, video shot, shots, shot took, shot, favorite shots, shoot, drone shot]   
2                    [heard gunshots, hear gunshots, gunshots fireworks, gunshots, heard shots, shootings, gunshot, shots fired, anyone else hear, gunfire]   
3                    [shooting, shooting shooting, shooting near, shootings, another shooting, gunshots, shooting outside, people shot, shot near, shooter]   
4   [sexual assault, accused sexual assault, sexually assault, sex assault, assaulted, accused sexual, charged assault, assault charges, assaults, assault]   
..                                                                                                                                                      ...   
89                                           [scooter stolen, stolen, scooter, stolen today, got stolen, theft, scooters, get stolen, moped, police report]   
90         [new orleans police, shot killed, people shot, killed shooting, orleans police, fatally shot, new orleans, shot dead, woman shot, shooting shot]   
91              [carjackings, chicago crime, carjacking, armed carjacking, chicago police, chicago police say, carjack, chicago, across chicago, robberies]   
92                          [violent crime rate, detroit police, violent crime, city crime, crime rate, crime, crime spike, crime new, detroit, crime plan]   
93                                                [new orleans, katrina, nola, louisiana, orleans, politicians, hurricanes, hurricane, memphis, mardi gras]   

                                                                                                                                                      MMR  \
0                                                                              [crime, like, shooting, murder, chicago, stolen, school, nyc, years, home]   
1  

In [32]:
# Show topic
topic_model.get_topic(14)

[('old', np.float64(0.11167292706331299)),
 ('year', np.float64(0.09342981468242907)),
 ('shot', np.float64(0.046676381278542234)),
 ('girl', np.float64(0.03926861208911103)),
 ('killed', np.float64(0.03843531700219713)),
 ('boy', np.float64(0.03840555376709412)),
 ('shooting', np.float64(0.037673827977572336)),
 ('detroit', np.float64(0.026951394637800528)),
 ('police', np.float64(0.022233713204972146)),
 ('woman', np.float64(0.021639949705360054))]

In [33]:
freq = topic_model.get_topic_info()

top_20 = freq#[1:21]
top_20 = top_20.explode('Representative_Docs')
#top_20

In [34]:
top_20 = top_20.rename(columns={'Name':'tweet_topic'})
#top_20

In [35]:
top_20_topics = list(set(top_20['Topic'].values))
top_20_topics = list(filter(lambda x: x != -1, top_20_topics))
#top_20_topics

In [ ]:
# here we are merging topics with main document

df_with_topics = pd.DataFrame({"Document":stored_sentences,
            "Topic":topic_model.topics_})
df_with_topics = df_with_topics.merge(freq,on='Topic', how='inner')
df_with_topics = df_with_topics.rename(columns={
    'Document': 'bertopic_preprocessed_text',
    'Topic': 'tweet_topic'
})

#df_with_topics.head()

In [37]:
df_tweet_with_topic = df.merge(df_with_topics,
                               left_on='cleaned_text',
                               right_on='bertopic_preprocessed_text',
                               how='left')

# Drop duplicates based on 'bertopic_preprocessed_text'
df_tweet_with_topic = df_tweet_with_topic.drop_duplicates(subset=['bertopic_preprocessed_text'])

df_tweet_with_topic.head()

author  created_utc      id  \
0         [deleted]   1546314346  abemmi   
5         [deleted]   1546314372  abemqc   
6         [deleted]   1546380318  abmjmw   
7  WildVelociraptor   1546439160  abtu2p   
8       Downbad2018   1546482755  ac0vud   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      selftext  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    [deleted]   
5                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
# we are now saving main df with topics and extracted keywords
df_tweet_with_topic = df_tweet_with_topic.rename(columns = {
    'Name' : 'tweet_topic_name'
})
df_tweet_with_topic['tweet_topic'] = df_tweet_with_topic['tweet_topic'].fillna('No data')
df_tweet_with_topic['tweet_topic_name'] = df_tweet_with_topic['tweet_topic_name'].fillna('Other')
df_tweet_with_topic['tweet_topic_name'] = df_tweet_with_topic.apply(lambda x:
                                                                    x['tweet_topic_name']
                                                                    if x['tweet_topic'] != -1
                                                                    else 'Other', axis = 1)

#df_tweet_with_topic

# Save the  posts with topics texts
output_file_path = 'topic/posts_with_topics.csv'
df_tweet_with_topic.to_csv(output_file_path, index=False)

In [39]:
df_tweet_with_topic.groupby('tweet_topic_name').count()

,author,created_utc,id,selftext,subreddit,title,date,crime,false_match,text,cleaned_text,topic,bertopic_preprocessed_text,tweet_topic,Count,Representation,KeyBERT,MMR,Representative_Docs
tweet_topic_name,,,,,,,,,,,,,,,,,,,
0_shot_shots_film_of,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457,3457
10_stabbed_death_man_fatally,741,741,741,741,741,741,741,741,741,741,741,741,741,741,741,741,741,741,741
11_me_was_he_my,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196
12_hate_asian_crime_crimes,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775
13_vaccine_covid_governor_health,602,602,602,602,602,602,602,602,602,602,602,602,602,602,602,602,602,602,602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90_carjackings_chicago_carjacking_cwb,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126
91_violent_crime_mayor_total,223,223,223,223,223,223,223,223,223,223,223,223,223,223,223,223,223,223,223
92_orleans_new_you_the,179,179,179,179,179,179,179,179,179,179,179,179,179,179,179,179,179,179,179


In [40]:
df_tweet_with_topic['tweet_topic_name'].value_counts()

tweet_topic_name
Other                                3668
0_shot_shots_film_of                 3457
1_gunshots_anyone_fireworks_heard    2168
2_shooting_shot_dead_mass            2032
3_assault_sexual_rape_sex            1430
                                     ... 
62_billion_murray_dorsey_trump        115
88_scooter_moped_stolen_scooters      112
85_lightfoot_lori_mayor_chicago       107
78_backs_bmurray_pres_int              44
53_theil_payoffs_obama_soros           41
Name: count, Length: 94, dtype: int64

In [41]:
summary_df = pd.DataFrame(df_tweet_with_topic['tweet_topic_name'].value_counts()).reset_index()
summary_df

,tweet_topic_name,count
0,Other,3668
1,0_shot_shots_film_of,3457
2,1_gunshots_anyone_fireworks_heard,2168
3,2_shooting_shot_dead_mass,2032
4,3_assault_sexual_rape_sex,1430
...,...,...
89,62_billion_murray_dorsey_trump,115
90,88_scooter_moped_stolen_scooters,112
91,85_lightfoot_lori_mayor_chicago,107
92,78_backs_bmurray_pres_int,44


In [42]:
freq = topic_model.get_topic_info()
freq = freq[['Topic', 'Name', 'Representation']]
summary_df = summary_df.rename(columns = {'tweet_topic_name': 'Name', 'count': 'Count'})
display(freq)
display(summary_df)

,Topic,Name,Representation
0,-1,-1_shooting_in_on_the,"[shooting, in, on, the, shot, at, for, to, crime, of]"
1,0,0_shot_shots_film_of,"[shot, shots, film, of, this, from, drone, some, the, my]"
2,1,1_gunshots_anyone_fireworks_heard,"[gunshots, anyone, fireworks, heard, shooting, hear, was, on, what, shots]"
3,2,2_shooting_shot_dead_mass,"[shooting, shot, dead, mass, injured, park, man, killed, wounded, at]"
4,3,3_assault_sexual_rape_sex,"[assault, sexual, rape, sex, man, raped, woman, charged, accused, arrested]"
...,...,...,...
89,88,88_scooter_moped_stolen_scooters,"[scooter, moped, stolen, scooters, it, vespa, my, riding, electric, if]"
90,89,89_orleans_new_quarter_french,"[orleans, new, quarter, french, bourbon, shot, nopd, police, shooting, street]"
91,90,90_carjackings_chicago_carjacking_cwb,"[carjackings, chicago, carjacking, cwb, armed, police, spree, robberies, een, in]"
92,91,91_violent_crime_mayor_total,"[violent, crime, mayor, total, detroit, fbi, rate, cities, city, homicides]"


,Name,Count
0,Other,3668
1,0_shot_shots_film_of,3457
2,1_gunshots_anyone_fireworks_heard,2168
3,2_shooting_shot_dead_mass,2032
4,3_assault_sexual_rape_sex,1430
...,...,...
89,62_billion_murray_dorsey_trump,115
90,88_scooter_moped_stolen_scooters,112
91,85_lightfoot_lori_mayor_chicago,107
92,78_backs_bmurray_pres_int,44


In [ ]:
# we are now creating summary table for further use

summary_df = summary_df.merge(freq, on='Name', how='left').fillna('Other')
summary_df = summary_df[['Topic', 'Name', 'Representation', 'Count']]

In [44]:
summary_df['Representation'] = summary_df['Representation'].apply(lambda x: 'Other' if x == 'Other' else ', '.join(x))
summary_df['Topic'] = summary_df['Topic'].apply(lambda x: -1 if x == 'Other' else int(x))

In [45]:
summary_df.head(50)

# Save the  summary resulsts
output_file_path = 'topic/summary.csv'
summary_df.to_csv(output_file_path, index=False)

In [ ]:
# this stage is useful if you are planning to make qualitative analysis
# the main idea is to extract top 20 representative text per topic
# it extracted based on their probability score

freq = topic_model.get_topic_info()

top_20 = freq#[1:21]
top_20 = top_20.explode('Representative_Docs')
top_20

Topic  Count                          Name  \
0      -1   3927         -1_shooting_in_on_the   
0      -1   3927         -1_shooting_in_on_the   
0      -1   3927         -1_shooting_in_on_the   
1       0   3545          0_shot_shots_film_of   
1       0   3545          0_shot_shots_film_of   
..    ...    ...                           ...   
92     91    247  91_violent_crime_mayor_total   
92     91    247  91_violent_crime_mayor_total   
93     92    179        92_orleans_new_you_the   
93     92    179        92_orleans_new_you_the   
93     92    179        92_orleans_new_you_the   

                                                                 Representation  \
0                         [shooting, in, on, the, shot, at, for, to, crime, of]   
0                         [shooting, in, on, the, shot, at, for, to, crime, of]   
0                         [shooting, in, on, the, shot, at, for, to, crime, of]   
1                     [shot, shots, film, of, this, from, drone, some, the, my]   
1                     [shot, shots, film, of, this, from, drone, some, the, my]   
..                                                                          ...   
92  [violent, crime, mayor, total, detroit, fbi, rate, cities, city, homicides]   
92  [violent, crime, mayor, total, detroit, fbi, rate, cities, city, homicides]   
93                          [orleans, new, you, the, to, that, it, and, we, of]   
93                          [orleans, new, you, the, to, that, it, and, we, of]   
93                          [orleans, new, you, the, to, that, it, and, we, of]   

                                                                                                                            KeyBERT  \
0                                             [community, chicago, neighborhood, home, nyc, public, back, houston, street, memphis]   
0                                             [community, chicago, neighborhood, home, nyc, public, back, houston, street, memphis]   
0                                             [community, chicago, neighborhood, home, nyc, public, back, houston, street, memphis]   
1                       [shot film, film shot, another shot, video shot, shots, shot took, shot, favorite shots, shoot, drone shot]   
1                       [shot film, film shot, another shot, video shot, shots, shot took, shot, favorite shots, shoot, drone shot]   
..                                                                                                                              ...   
92  [violent crime rate, detroit police, violent crime, city crime, crime rate, crime, crime spike, crime new, detroit, crime plan]   
92  [violent crime rate, detroit police, violent crime, city crime, crime rate, crime, crime spike, crime new, detroit, crime plan]   
93                        [new orleans, katrina, nola, louisiana, orleans, politicians, hurricanes, hurricane, memphis, mardi gras]   
93                        [new orleans, katrina, nola, louisiana, orleans, politicians, hurricanes, hurricane, memphis, mardi gras]   
93                        [new orleans, katrina, nola, louisiana, orleans, politicians, hurricanes, hurricane, memphis, mardi gras]   

                                                                                                                                          MMR  \
0                                                                  [crime, like, shooting, murder, chicago, stolen, school, nyc, years, home]   
0                                                                  [crime, like, shooting, murder, chicago, stolen, school, nyc, years, home]   
0                                                                  [crime, like, shooting, murder, chicago, stolen, school, nyc, years, home]   
1                                               [film, video, photos, skyline, photography, sunset, favorite, drone shot, shoot, music video]   
1                                               [film, video

In [47]:
top_20 = top_20.rename(columns={'Name':'tweet_topic'})
#top_20

In [48]:
top_20_samples = []

for topic in top_20_topics:
    samples = df_tweet_with_topic.loc[df_tweet_with_topic['tweet_topic'] == topic ]
    samples = samples[['tweet_topic', 'text']]
    samples = samples.drop_duplicates(subset=['text'])
    samples = samples.rename(columns = {
        'tweet_topic': 'Topic',
        'text': 'Representative_Docs'
    })
#     samples = samples.sample(n=min(samples.shape[0], 20), random_state=1)
    samples = samples.sample(n=20, random_state=1)

    top_20_samples.append(samples)

In [49]:
top_20_samples_df = pd.concat(top_20_samples).reset_index().drop(columns='index')
top_20_samples_df

,Topic,Representative_Docs
0,0,"I was going over my archives, and I had shots to make a panorama of the Mid-Wilshire area from Aug 2017. Enjoy!:"
1,0,"Urgently seeking a photographer: I had a photographer lined up for tomorrow 9/24 and Saturday 9/25 that had to cancel due to COVID. So I’m reaching out to see if any one here can help! \n\n\nThe photoshoot tomorrow was supposed to be candid shots of myself, fiancee, and the officiator from 11-1 in Freedom Park. The second shoot was for the ceremony/reception from 6-8.\n\nAnyways, if you know someone that can help, send a pm!"
2,0,My Shot of Wilshire Grand and Central Library:
3,0,Dashcam shot of someone crossing Alameda with a Chucky doll dressed as their baby:
4,0,LACLEDE’S LANDING: Hope he has his tetanus shot:
...,...,...
1855,92,"A city affair to continue or end?: I've been cheating on my home of Chicago with New Orleans for years. 2-4 times a year I've been visiting. I'll never Air Bnb cause I see what it does to the neighborhoods . Was so weird to see tourists flooding the streets of Bywater one of my last visits. I can retire in a few years and think about making things more permanent but I read your threads and see the news and my passion is struggling with caution. Violence I'm sadly used to but I would like to not be. basic amenities and public service seem to be a huge challenge. I love you people of New Orleans and it's the only place I ever feel at home, but is it worth it? No disrespect intended. I very much respect the people and the spirit of New Orleans. Also I'm not sure how safe my daca wife is in such a red state. Be safe and well in 2023"
1856,92,"Downtown's deterioration feels like it's getting worse, faster. Anyone else considering getting out?: The homeless encroachments, violence, shuttered businesses, downtown LA feels like it's getting worse and worse daily and it's got me questioning if I should move out of the area. The value here is pretty great during non-pandemic times, compared to the other major centers of food/bars/entertainment, but this all has me questioning if I should bite the bullet and move to Culver or Glendale at this point.\n\nAnyone else?"
1857,92,"Today in therapy, I talked about my experience during Hurricane Katrina as a ""Katrina Baby."" I talked about everything for the first time in 17 years and it came out as word vomit. It never occurred me that what we went through would be considered ""trauma."": Really, I just spilled my guts this morning. Just one long trauma dump. Didn't realize how much I had suppressed. \n\nThis came months after my therapist first brought up Hurricane Katrina with me (we got on the subject somehow). The crazy part about it? She asked how I dealt with the trauma. My response was: ""Trauma? I didn't go through any trauma."" I mean, I was lucky--we evacuated. My house wasn't damaged. I was fine. \n\nBut then, I realized--the anxiety that weekend before the storm. I never saw my mom being so anxious about a hurricane. Spending two months in Baton Rouge with my sister with 8 other people in a 1 bedroom apartment. My mom getting sick and having to take care of her. Hearing so many stories from refugees that survived the floods and said they couldn't find loved ones (people breaking down crying in stores). My oldest sister losing her house and then her and her kids had to move to Alabama (after years of them visiting our home at least once a week--I grew up with my nieces and nephews). Losing friends that had to stay elsewhere and going through school alone. Just seeing the city underwater and hearing and seeing people trapped in attics or on the roof or dying in the Superdome. So many horror stories for the next...years. Just coming home and seeing the city damaged and different. Seeing those damn signs showing ""1 dead in attic."" The National Guard in the city, waiting in lines outside grocery and lines being guarded by armed soldiers. That one time driving with my mom and seeing men in hazmat suits carry

In [50]:
top_20 = top_20.drop(columns=['Representative_Docs']).drop_duplicates(subset=['Topic'])
top_20_updated = top_20.merge(top_20_samples_df, on='Topic')
#top_20_updated

In [51]:

# Save the representative texts
output_file_path = 'topic/representative_texts.csv'
top_20_updated.to_csv(output_file_path, index=False)

# Topic Timeline

In [52]:
# if you do not use probabilities, topics some withing tuple, here I get it as list
# freq.Count.sum() # test the distribution
#Topics = topics[0]
#print(len(Topics))

print(len(topics))
print(len(stored_timestamp))
print(len(stored_sentences))

51429
51429
51429


In [53]:
topics_over_time = topic_model.topics_over_time(docs=stored_sentences,
                                                #topics=topics,
                                                timestamps=stored_timestamp,
                                                global_tuning=True,
                                                evolution_tuning=True,
                                                nr_bins=20)


# graph
fig = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10, width=1000)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.02
),
legend_font_size=10)

fig.show()




20it [00:08,  2.27it/s]


In [54]:
# fig.write_html("run_visual/"+ran_dict[filename]+"MaliciousBot-Misinfomation_topics over time (All 8 - good).html")
fig.write_html("figures/topics_over_time.html")

# fig.show()

In [55]:
# visualize certain topics

fig = topic_model.visualize_topics_over_time(topics_over_time, topics=[ 2, 3, 4,5, 6, 7,8, 9], width=1000)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.02
),
legend_font_size=10)

fig.show()


In [56]:
# fig.write_html("run_visual/"+ran_dict[filename]+"MaliciousBot-Misinfomation_topics over time (All 8 - good).html")
fig.write_html("figures/topics_over_time_selected_topics.html")

# fig.show()

### Cohorence score

In [ ]:
# Sometimes journals ask for cohorence score for Bertopic, it is a rare instances but it could happen
# gensim package is outdated now, not  compatable with transformers
# I modified these codes , depending on the python environment, I need to upgrade or downgrade numpy 
# it is a working code block but I suggest you to create new environment to run this codes
#  because you need to downgrade several packages that come with BERTopic
# to retain your BERTOpic environment, it would be better to  create a new environment and
#  install gensim package and run the codes with -stored_sentences-  that come above

import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel


# Ensure stored_sentences and topics have the same length
if len(stored_sentences) != len(topics):
    raise ValueError("stored_sentences and topics must have the same length")

# Create DataFrame
documents = pd.DataFrame({
    "Document": stored_sentences,
    "ID": range(len(stored_sentences)),
    "Topic": topics
})

# Group by topic and join documents
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})

# Preprocess cleaned documents
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract features for Topic Coherence evaluation
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [words for topic in range(len(set(topics))-1)]

# Flatten topic_words and convert to IDs
topic_words_ids = []
for topic in topic_words:
    topic_ids = [dictionary.token2id[word] for word in topic if word in dictionary.token2id]
    topic_words_ids.append(topic_ids)

# Evaluate Coherence
coherence_model = CoherenceModel(topics=topic_words_ids,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
print('Coherence value of the model:', coherence)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject